# Notebook 5 : Web scraping

In [ ]:
# Décommenter la ligne suivante pour installer lxml (nécessaire pour read_html)
# %pip install lxml

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import requests

from bs4 import BeautifulSoup

## Bonsaïs

Le site [Umi Zen Bonsai](https://umizenbonsai.com/) est une boutique de vente en ligne dédiée aux bonsaïs. Les conifères sont disponible sur la page web [https://umizenbonsai.com/shop/bonsai/coniferes/](https://umizenbonsai.com/shop/bonsai/coniferes/). Comme beaucoup d'autres sites, l'information est organisée en blocs dans lesquels il est possible de récupérer des données.

Pour scraper ce type de site, le processus consiste à capturer les blocs dans un premier temps, puis à en extraire les données.

1. Récupérer le contenu de la page avec `requests` et passer le résultat au parser de `BeautifulSoup`.

In [3]:
url_bonsais = "https://umizenbonsai.com/shop/bonsai/coniferes/"

r_bonzais = requests.get(url_bonsais)
assert r_bonzais.status_code == 200, f"Erreur {r_bonzais.status_code}"

soup_bonzais = BeautifulSoup(r_bonzais.text, "html.parser")

print(soup_bonzais.prettify()[:1000])  # Affiche les 1000 premiers caractères du HTML

<!DOCTYPE html>
<html class="html" lang="fr-FR">
 <head>
  <meta charset="utf-8"/>
  <script>
   if(navigator.userAgent.match(/MSIE|Internet Explorer/i)||navigator.userAgent.match(/Trident\/7\..*?rv:11/i)){var href=document.location.href;if(!href.match(/[?&]nowprocket/)){if(href.indexOf("?")==-1){if(href.indexOf("#")==-1){document.location.href=href+"?nowprocket=1"}else{document.location.href=href.replace("#","?nowprocket=1#")}}else{if(href.indexOf("#")==-1){document.location.href=href+"&nowprocket=1"}else{document.location.href=href.replace("#","&nowprocket=1#")}}}}
  </script>
  <script>
   (()=>{class RocketLazyLoadScripts{constructor(){this.v="2.0.3",this.userEvents=["keydown","keyup","mousedown","mouseup","mousemove","mouseover","mouseenter","mouseout","mouseleave","touchmove","touchstart","touchend","touchcancel","wheel","click","dblclick","input","visibilitychange"],this.attributeEvents=["onblur","onclick","oncontextmenu","ondblclick","onfocus","onmousedown","onmouseenter","onmo

2. Écrire un sélecteur CSS pour capturer les éléments `li` qui contiennent les blocs correspondants aux bonsaïs. Vérifier sur le site que le nombre de bonsaïs affichés correspond.

In [13]:
selector_blocs = "li.entry"
bonsais_blocs = soup_bonzais.select(selector_blocs)
print(f"Nombre de bonsais : {len(bonsais_blocs)}")

Nombre de bonsais : 9


3. Écrire une fonction qui prend un bloc de la liste précédente et retourne un tuple contenant le nom, le prix et le lien de description du bonsaï.

In [44]:
def extract_bonsai_info(bonsai_bloc):
    """Extrait les informations d'un bonsaï à partir de son bloc HTML."""
    bonsai_info = {}
    
    # Nom du bonsaï
    name_link=bonsai_bloc.select("li.title > h2 > a")
    bonsai_info["nom"]=name_link[0].text
    
    # Prix
    prices=bonsai_bloc.select("span.price span bdi")
    bonsai_info["price"]=prices[-1].text.replace("€", "").replace(",", "")
    
    # Lien vers la page du produit
    bonsai_info["link"]=name_link[0].attrs["href"]
    
    return bonsai_info

for b in bonsais_blocs:
    bonsai_info = extract_bonsai_info(b)
    print(bonsai_info)  # Affiche les informations de chaque bonsaï

{'nom': 'Bonsai Cyprès du Japon – 33cm', 'price': '140.00', 'link': 'https://umizenbonsai.com/vente/bonsai-cypres-du-japon/'}
{'nom': 'Bonsai Genévrier de Phénicie – 59cm', 'price': '225.00', 'link': 'https://umizenbonsai.com/vente/bonsai-genevrier-de-phenicie/'}
{'nom': 'Bonsai Genévrier de Phénicie – 67cm', 'price': '333.00', 'link': 'https://umizenbonsai.com/vente/bonsai-genevrier-phenicie/'}
{'nom': 'Bonsai If Commun – 58cm', 'price': '1190.00', 'link': 'https://umizenbonsai.com/vente/bonsai-if-commun-2/'}
{'nom': 'Bonsai If Commun – 85cm', 'price': '585.00', 'link': 'https://umizenbonsai.com/vente/bonsai-if-commun/'}
{'nom': 'Bonsai If du Japon – 50cm', 'price': '750.00', 'link': 'https://umizenbonsai.com/vente/bonsai-if-du-japon/'}
{'nom': 'Bonsai If du Japon – 63cm', 'price': '1290.00', 'link': 'https://umizenbonsai.com/vente/bonsai-if-japon/'}
{'nom': 'Bonsai Pin Sylvestre – 38cm', 'price': '440.00', 'link': 'https://umizenbonsai.com/vente/bonsai-pin-sylvestre-3/'}
{'nom': 'Bon

4. Utiliser les deux questions précédentes pour construire un dataframe contenant les données des bonsaïs.

5. (*Bonus*) Écrire une fonction pour récupérer la provenance, le feuilage et les dimension du bonsaï à partir du lien de description. Utiliser cette fonction pour ajouter des colonnes au dataframe précédent

## Trampoline

Le trampoline est un sport olympique depuis les jeux de Sydney en 2000. La page suivante contient les listes des hommes et des femmes ayant obtenu une médaille olympique dans cette discipline :
[https://fr.wikipedia.org/wiki/Liste_des_m%C3%A9daill%C3%A9s_olympiques_au_trampoline](https://fr.wikipedia.org/wiki/Liste_des_m%C3%A9daill%C3%A9s_olympiques_au_trampoline)

Un tableau est contenu dans un élément `table` avec des balises pour les lignes `tr`, pour les colonnes `th`, pour les cellules `td`, ... Cela peut être fastidieux à scraper et très répétitif. Heureusement, Pandas propose la fonction `read_html` pour récupérer des tableaux sous forme de dataframes à partir d'une page web.

1. Utiliser la fonction `read_html` de Pandas sur la page des médaillés olympiques au trampoline. Combien de dataframes sont récupérés ?

2. Extraire de la liste précédente les dataframes des médailles masculines et féminines.

3. À partir de ces dataframes, compter combien chaque pays a reçu de médailles d'or, d'argent et de bronze.

4. (*Bonus*) Construire un dataframe contenant, pour chaque pays, le nombre de médailles d'or, d'argent et de bronze ainsi que le nombre total de médailles. Classer ce dataframe dans l'ordre usuel en fonction d'abord du nombre de médailles d'or, puis du nombre de médailles d'argent et enfin du nombre de médailles de bronze. Comparer le résultat avec le tableau des médailles sur la page [https://fr.wikipedia.org/wiki/Trampoline_aux_Jeux_olympiques](https://fr.wikipedia.org/wiki/Trampoline_aux_Jeux_olympiques).

## Cate Blanchett

Dans le cours, nous avons essayé de trouver avec quels acteurs Cate Blanchett a joué le plus au cours des années 2000. Pour cela, nous avons récupéré la liste des pages Wikipedia des films où elle tient un rôle avec le code suivant :

In [53]:
url_wikipedia = "https://fr.wikipedia.org"
url_blanchett = url_wikipedia + "/wiki/Cate_Blanchett"

r_blanchett = requests.get(url_blanchett)
assert r_blanchett.status_code == 200, f"Erreur {r_blanchett.status_code}"

soup_blanchett = BeautifulSoup(r_blanchett.text, "html.parser")

selector_films = "#mw-content-text div ul:nth-of-type(3) li i a"
films_blanchett = soup_blanchett.select(selector_films)

films_data = [
    {
        "titre": film.attrs["title"],
        "url_wikipedia": url_wikipedia + film.attrs["href"]
    }
    for film in films_blanchett
    if not (
        film.attrs.get("class") == ["new"] # Film sans page
        or film.attrs["title"] == "Galadriel" # Mauvais lien
    )
]

films = pd.DataFrame(films_data)

films

,titre,url_wikipedia
0,Les Larmes d'un homme,https://fr.wikipedia.org/wiki/Les_Larmes_d%27u...
1,Intuitions,https://fr.wikipedia.org/wiki/Intuitions
2,"Bandits (film, 2001)","https://fr.wikipedia.org/wiki/Bandits_(film,_2..."
3,Le Seigneur des anneaux : La Communauté de l'a...,https://fr.wikipedia.org/wiki/Le_Seigneur_des_...
4,Charlotte Gray,https://fr.wikipedia.org/wiki/Charlotte_Gray
5,Terre Neuve (film),https://fr.wikipedia.org/wiki/Terre_Neuve_(film)
6,"Heaven (film, 2002)","https://fr.wikipedia.org/wiki/Heaven_(film,_2002)"
7,Le Seigneur des anneaux : Les Deux Tours,https://fr.wikipedia.org/wiki/Le_Seigneur_des_...
8,Veronica Guerin (film),https://fr.wikipedia.org/wiki/Veronica_Guerin_...
9,Coffee and Cigarettes,https://fr.wikipedia.org/wiki/Coffee_and_Cigar...


Le sélecteur CSS que nous avons utilisé ne permettait pas d'obtenir la réponse à notre question car il ne capturait pas toutes les listes d'acteurs (organisation différente pour *Coffee and Cigarettes*, double colonne pour *Aviator*, ...). En effet, les pages Wikipedia des films ne sont pas uniformes et il n'est pas possible d'extraire la distribution de tous les films avec le même sélecteur.

Pour remédier à cela, nous proposons ici d'aller scraper la liste des acteurs sur le site [TMDB](https://www.themoviedb.org/) (*The Movie Database*) dont les pages obéissent toutes à la même organisation. Les pages Wikipedia relatives à des films contiennent toutes un lien externe vers ce site.

1. Pour chaque film, scraper la page Wikipedia pour récupérer le lien vers la page TMDB associée et déduire le lien du casting complet qui ser ajouté le dans une nouvelle colonne du dataframe `films`.

In [66]:
def get_tmdb_url(url):
    r_wm = requests.get(url)
    assert r_wm.status_code == 200, f"Erreur {r_wm.status_code}"
    soup_wm = BeautifulSoup(r_wm.text)
    urls_tmdb = soup_wm.select("a.external")
    for u in urls_tmdb:
        if u.text=="The Movie Database":
            url_tmdb = u.attrs["href"]
            break
    url_final= url_tmdb+"/cast"
    return url_final

get_tmdb_url(films.iloc[1].url_wikipedia)

'https://www.themoviedb.org/movie/2046/cast'

2. La liste des acteurs d'un film se présente comme une liste ordonnée `ol` dans les pages TMDB. Scraper les pages de casting pour ajouter la liste des acteurs de chaque film dans une nouvelle colonne du dataframe `films`.

In [75]:
def get_tmdb_cast(url):
    r_tmdb = requests.get(url)
    assert r_tmdb.status_code == 200, f"Erreur {r_tmdb.status_code}"
    soup_tmdb = BeautifulSoup(r_tmdb.text)
    casthtml = soup_tmdb.select("section.panel:nth-of-type(1) ol li div div p a")
    cast=[]
    for c in casthtml:
        cast.append(c.text)
    return cast

films["url_tmdb_casting"] = films.url_wikipedia.apply(get_tmdb_url)
films["acteurs"] = films.url_tmdb_casting.apply(get_tmdb_cast)

print(f"total cast = {len(films.iloc[0].acteurs)}")
print(films.iloc[0].acteurs)

total cast = 65
['Christina Ricci', 'Johnny Depp', 'Cate Blanchett', 'John Turturro', 'Harry Dean Stanton', 'Oleg Yankovskiy', 'Don Fellows', 'Claudia Lander-Duke', 'Danny Scheinmann', 'Anna Tzelniker', 'Barry Davis', 'Thom Osborn', 'Frank Chersky', 'Daniel Hart', 'Peter Majer', 'Hana Maria Pravda', 'Ayala Meir', 'Abraham Hassan', 'Lloyd Martin', 'Uri Meir', 'Sophie Richman', 'Theo Wishart', 'Michael Mount', 'Harry Flinder', 'Danny Richman', 'Victor Sobchak', 'Sue Cleaver', 'Clifford Barry', 'Paul Clayton', 'Diana Hoddinott', 'Richard Albrecht', 'Ornella Bryant', 'Sam Friend', 'Isabella Melling', 'Alan David', 'Imogen Claire', 'Miriam Karlin', 'Consuelo De Haviland', 'Katia Labèque', 'Marielle Labèque', 'George Antoni', 'Pablo Verón', 'Taraf de Haidouks', 'Odile Roire', 'Brigitte Boucher', 'Norah Krief', 'Hélène Hardouin', 'Hugues Dalmagro', 'Cedric Gary', 'Saïfi Ghoul', 'Manfred Andrae', 'Richard Sammel', 'Zirek', 'Joyce Springer', 'Cyril Shaps', 'Anna Korwin', 'Mark Ivanir', 'Alfred 

In [85]:
for _, film in films.iterrows():
    print(film.titre)
    print(f"total cast = {len(film.acteurs)}")
    print(film.acteurs)

Les Larmes d'un homme
total cast = 65
['Christina Ricci', 'Johnny Depp', 'Cate Blanchett', 'John Turturro', 'Harry Dean Stanton', 'Oleg Yankovskiy', 'Don Fellows', 'Claudia Lander-Duke', 'Danny Scheinmann', 'Anna Tzelniker', 'Barry Davis', 'Thom Osborn', 'Frank Chersky', 'Daniel Hart', 'Peter Majer', 'Hana Maria Pravda', 'Ayala Meir', 'Abraham Hassan', 'Lloyd Martin', 'Uri Meir', 'Sophie Richman', 'Theo Wishart', 'Michael Mount', 'Harry Flinder', 'Danny Richman', 'Victor Sobchak', 'Sue Cleaver', 'Clifford Barry', 'Paul Clayton', 'Diana Hoddinott', 'Richard Albrecht', 'Ornella Bryant', 'Sam Friend', 'Isabella Melling', 'Alan David', 'Imogen Claire', 'Miriam Karlin', 'Consuelo De Haviland', 'Katia Labèque', 'Marielle Labèque', 'George Antoni', 'Pablo Verón', 'Taraf de Haidouks', 'Odile Roire', 'Brigitte Boucher', 'Norah Krief', 'Hélène Hardouin', 'Hugues Dalmagro', 'Cedric Gary', 'Saïfi Ghoul', 'Manfred Andrae', 'Richard Sammel', 'Zirek', 'Joyce Springer', 'Cyril Shaps', 'Anna Korwin', '

3. Utiliser le résultat de la question précédente pour répondre à la question initiale : avec quels acteurs Cate Blanchett a-t-elle partagé l'affiche le plus souvent au cours des années 2000 ?

In [84]:
films["acteurs"].explode().value_counts()

Cate Blanchett     23
Peter Jackson       4
Hugo Weaving        4
Billy Boyd          3
Sala Baker          3
                   ..
Ray McKinnon        1
Simon Baker         1
Jay Tavare          1
Steve Reevis        1
Michael Wozniak     1
Name: acteurs, Length: 1245, dtype: int64